In [1]:
# Preparing the environment
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

In [ ]:
import torch
import numpy as np
import smplx

from human_body_prior.tools.omni_tools import copy2cpu as c2c
from os import path as osp

support_dir = './data'

In [3]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
comp_device

device(type='cpu')

In [4]:
amass_npz_fname = osp.join(support_dir, 'YOGI_2_latest_smplx_neutral/train/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.npz') # the path to body data
bdata = np.load(amass_npz_fname)

# you can set the gender manually and if it differs from data's then contact or interpenetration issues might happen
subject_gender = bdata['gender']

print('Data keys available:%s'%list(bdata.keys()))

print('The subject of the mocap sequence is  {}.'.format(subject_gender))


Data keys available:['gender', 'surface_model_type', 'mocap_frame_rate', 'mocap_time_length', 'trans', 'poses', 'betas', 'num_betas', 'root_orient', 'pose_body', 'pose_jaw', 'pose_eye', 'pose_hand']
The subject of the mocap sequence is  neutral.


In [5]:
from human_body_prior.body_model.body_model import BodyModel

bm_fname = osp.join(support_dir, 'body_models/smplh/{}/model.npz'.format(subject_gender))

num_betas = 16 # number of body parameters

bm = BodyModel(f'{support_dir}/models_lockedhead/smplx/SMPLX_NEUTRAL.npz', 'smplx', num_betas=num_betas).to(comp_device)
faces = c2c(bm.f)

In [6]:
time_length = len(bdata['trans'])

body_parms = {
    'root_orient': torch.Tensor(bdata['poses'][:, :3]).to(comp_device), # controls the global root orientation
    'pose_body': torch.Tensor(bdata['poses'][:, 3:66]).to(comp_device), # controls the body
    'pose_hand': torch.Tensor(bdata['poses'][:, 66:]).to(comp_device), # controls the finger articulation
    'trans': torch.Tensor(bdata['trans']).to(comp_device), # controls the global body position
    'betas': torch.Tensor(np.repeat(bdata['betas'][:num_betas][np.newaxis], repeats=time_length, axis=0)).to(comp_device), # controls the body shape. Body shape is static
    # 'dmpls': torch.Tensor(bdata['dmpls'][:, :num_dmpls]).to(comp_device) # controls soft tissue dynamics
}

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in body_parms.items()])))
print('time_length = {}'.format(time_length))


Body parameter vector shapes: 
root_orient: torch.Size([892, 3]) 
pose_body: torch.Size([892, 63]) 
pose_hand: torch.Size([892, 99]) 
trans: torch.Size([892, 3]) 
betas: torch.Size([892, 16])
time_length = 892


In [7]:
mosh = f"{support_dir}/mosh/train/220923_yogi_body_hands_03596_Boat_Pose_or_Paripurna_Navasana_-a_stageii.pkl"

In [8]:
import pickle as pkl
from moyo.eval.com_evaluation import smplx_to_mesh, visualize_mesh
from moyo.utils.constants import frame_select_dict_combined as frame_select_dict
import trimesh

# all_pose_names = [pp_name.replace('_stageii', '') for pp_name in pp_names]
selected_frame = frame_select_dict['_'.join(mosh.split('_')[5:-1])]

pp_pkl = mosh
pp_params = pkl.load(open(pp_pkl, 'rb'))
# read height, weight, gender meta
height = None
weight = None
gender = 'neutral'
subj_id = 'unknown'
pp_body_model_output, smplx_params, transls, faces = smplx_to_mesh(pp_params, f'{support_dir}/models_lockedhead/smplx/SMPLX_NEUTRAL.npz','smplx', gender=gender)

pp_mesh = visualize_mesh(pp_body_model_output, faces, frame_id=550)

In [9]:
scene = trimesh.scene.scene.Scene(pp_mesh)
scene.show()

In [10]:
scene.set_camera(angles=(2,2,2), distance=5)
scene.show()

In [12]:
scene = None

In [17]:
import pickle as pkl
from moyo.eval.com_evaluation import smplx_to_mesh, visualize_mesh
from moyo.utils.constants import frame_select_dict_combined as frame_select_dict
import trimesh

accad_npz = osp.join(support_dir, 'ACCAD/Male2MartialArtsKIcks_c3d/G2_-_Sidekick_leading_left_stageii.npz') 

pp_pkl = accad_npz
pp_params = np.load(accad_npz)
# read height, weight, gender meta
height = None
weight = None
gender = 'neutral'
subj_id = 'unknown'
pp_body_model_output, smplx_params, transls, faces = smplx_to_mesh(pp_params, f'{support_dir}/models_lockedhead/smplx/SMPLX_MALE.npz','smplx', gender=gender)

pp_mesh = visualize_mesh(pp_body_model_output, faces, frame_id=10)
scene = trimesh.scene.scene.Scene(pp_mesh)
scene.set_camera(angles=(2,2,2), distance=5)
scene.show()

In [ ]:
scene.save_image()

2025-04-14 21:55:53.924 python[26965:61958414] +[IMKClient subclass]: chose IMKClient_Modern
2025-04-14 21:55:53.970 python[26965:61958414] +[IMKInputSession subclass]: chose IMKInputSession_Modern


b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x0e\x10\x00\x00\x07\xa6\x08\x06\x00\x00\x00D\xb1\x1a\x81\x00\x01\x00\x00IDATx\x9c\xec\xdb\x01\t\x00 \x10\xc0@\xb5\x7f\xe7\xb7\x84 \x8c\xbb\x04+\xb0=3\xb3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x94\xf3;\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00x\xcf@\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00A\x06B\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x082\x10\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x90\x81\x10\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x82\x0c\x84\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x10d \x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80 \x03!\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x19\x08\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \xc8@\x08\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00A\x06B\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x082\x10\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x90

: 